In [1]:
%matplotlib inline
import os
import glob
import csv
import re
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import pywt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from mpl_toolkits.mplot3d import Axes3D 
from scipy import signal
from scipy.signal import find_peaks
from scipy import optimize
from scipy.stats import linregress, stats
import seaborn as sns

import datetime as dt
from textwrap import wrap


# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
os.getcwd()

'C:\\Users\\bina_\\Dropbox\\Robina Shaheen_PEMS2022\\PEMS_NOx_BC_CO2_ALPR\\ii.dwt'

In [2]:
#df_CO2_BC = pd.read_csv('./Results2/merged_CO2_BC_indexed.csv', index_col=0)
df_lic = pd.read_csv('./Results2/simplified_ALPR.csv', index_col=0)
df_CO2_BC = pd.read_csv('./Results2/merged_CO2_BC_EF_time_min.csv', index_col=0)
print(df_CO2_BC.head())
print(df_lic.head)
# print(dtypes(df_lic))


   time2_x  BC (ug/m3)   CO2 (ug/m3)  BC/CO2 EF/kg fuel                 time  \
0  0:05:40  193.393660  83812.264400           7.276122  2022-11-27 00:05:40   
1  0:08:36   94.484616  25506.239870          11.680998  2022-11-27 00:08:36   
2  0:10:50  159.601288  75172.331670           6.694893  2022-11-27 00:10:50   
3  0:15:07  249.352208  46814.807230          16.795590  2022-11-27 00:15:07   
4  0:17:18   83.091961   7618.485284          34.391860  2022-11-27 00:17:18   

   seconds    minutes  
0       45   5.666667  
1       44   8.600000  
2       60  10.833333  
3       22  15.116667  
4       35  17.300000  
<bound method NDFrame.head of                      Time     time2     license vehicle-make-name
0     2022-09-02 00:00:19  00:00:19  2102015672         chevrolet
1     2022-09-02 00:48:47  00:48:47  2104921995             volvo
2     2022-09-02 00:48:56  00:48:56  2104932924              ford
3     2022-09-02 00:49:11  00:49:11  2104946538         chevrolet
4     2022-09-0

In [3]:
print(df_lic.dtypes)
print(len(df_lic))

Time                 object
time2                object
license               int64
vehicle-make-name    object
dtype: object
1738


In [4]:
df_lic.rename(columns={'time2':'t1_indexed'}, inplace=True)
# df2.rename(columns={'index':'t1_indexed'}, inplace=True)
# df1.columns
print(df_lic.dtypes)
df_lic.info()

Time                 object
t1_indexed           object
license               int64
vehicle-make-name    object
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1738 entries, 0 to 1737
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Time               1738 non-null   object
 1   t1_indexed         1738 non-null   object
 2   license            1738 non-null   int64 
 3   vehicle-make-name  1738 non-null   object
dtypes: int64(1), object(3)
memory usage: 67.9+ KB


In [5]:
df_lic[['t1_indexed']] = df_lic[['t1_indexed']].apply(pd.to_datetime)
# #.apply(lambda x: x.value)
print(df_lic.dtypes)
df_lic.head()

Time                         object
t1_indexed           datetime64[ns]
license                       int64
vehicle-make-name            object
dtype: object


,Time,t1_indexed,license,vehicle-make-name
0,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet
1,2022-09-02 00:48:47,2022-11-27 00:48:47,2104921995,volvo
2,2022-09-02 00:48:56,2022-11-27 00:48:56,2104932924,ford
3,2022-09-02 00:49:11,2022-11-27 00:49:11,2104946538,chevrolet
4,2022-09-02 00:49:11,2022-11-27 00:49:11,2104947077,kenworth


In [8]:

df_lic['t1_indexed'] = pd.to_datetime(df_lic['t1_indexed'])
df_lic['seconds'] = df_lic['t1_indexed'].dt.hour * 60 + df_lic['t1_indexed'].dt.minute + df_lic['t1_indexed'].dt.second
df_lic['minutes'] = df_lic['t1_indexed'].dt.hour * 60 + df_lic['t1_indexed'].dt.minute + df_lic['t1_indexed'].dt.second/60
print(df_lic.dtypes)
df_lic


Time                         object
t1_indexed           datetime64[ns]
license                       int64
vehicle-make-name            object
seconds                       int64
minutes                     float64
dtype: object


,Time,t1_indexed,license,vehicle-make-name,seconds,minutes
0,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19,0.316667
1,2022-09-02 00:48:47,2022-11-27 00:48:47,2104921995,volvo,95,48.783333
2,2022-09-02 00:48:56,2022-11-27 00:48:56,2104932924,ford,104,48.933333
3,2022-09-02 00:49:11,2022-11-27 00:49:11,2104946538,chevrolet,60,49.183333
4,2022-09-02 00:49:11,2022-11-27 00:49:11,2104947077,kenworth,60,49.183333
...,...,...,...,...,...,...
1733,2022-09-02 23:20:55,2022-11-27 23:20:55,2186051819,dodge,1455,1400.916667
1734,2022-09-02 23:45:44,2022-11-27 23:45:44,2187540484,chevrolet,1469,1425.733333
1735,2022-09-02 23:45:56,2022-11-27 23:45:56,2187552584,kenworth,1481,1425.933333
1736,2022-09-02 23:46:24,2022-11-27 23:46:24,2187580106,ford,1450,1426.400000


In [9]:
df_CO2_BC.sort_values(by='minutes', inplace=True)
df_lic.sort_values(by='minutes', inplace=True)

In [20]:
df3= pd.merge_asof(
    df_CO2_BC, df_lic, on="minutes",direction="nearest",  
        allow_exact_matches=False)
df3.to_csv('./Results2/merged_CO2_BC_EF_ALPR_min.csv')

df3.head()

,time2_x,BC (ug/m3),CO2 (ug/m3),BC/CO2 EF/kg fuel,time,seconds_x,minutes,Time,t1_indexed,license,vehicle-make-name,seconds_y
0,0:05:40,193.393660,83812.264400,7.276122,2022-11-27 00:05:40,45,5.666667,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19
1,0:08:36,94.484616,25506.239870,11.680998,2022-11-27 00:08:36,44,8.600000,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19
2,0:10:50,159.601288,75172.331670,6.694893,2022-11-27 00:10:50,60,10.833333,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19
3,0:15:07,249.352208,46814.807230,16.795590,2022-11-27 00:15:07,22,15.116667,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19
4,0:17:18,83.091961,7618.485284,34.391860,2022-11-27 00:17:18,35,17.300000,2022-09-02 00:00:19,2022-11-27 00:00:19,2102015672,chevrolet,19


In [21]:
df4= df3.loc[df3['BC/CO2 EF/kg fuel'] >= 100]
df4.to_csv('./Results2/merged_CO2_BC_EF100_min.csv')
df4.head()

,time2_x,BC (ug/m3),CO2 (ug/m3),BC/CO2 EF/kg fuel,time,seconds_x,minutes,Time,t1_indexed,license,vehicle-make-name,seconds_y
11,0:35:22,155.293722,4221.974694,115.985464,2022-11-27 00:35:22,57,35.366667,2022-09-02 00:48:47,2022-11-27 00:48:47,2104921995,volvo,95
15,0:42:18,132.474562,2801.542226,149.107885,2022-11-27 00:42:18,60,42.300000,2022-09-02 00:48:47,2022-11-27 00:48:47,2104921995,volvo,95
26,1:06:46,166.373958,710.725389,738.157115,2022-11-27 01:06:46,112,66.766667,2022-09-02 01:09:23,2022-11-27 01:09:23,2106159630,kenworth,92
34,1:25:41,128.097471,3876.477391,104.200210,2022-11-27 01:25:41,126,85.683333,2022-09-02 01:11:41,2022-11-27 01:11:41,2106296684,chevrolet,112
39,1:36:34,44.829109,1390.088980,101.691065,2022-11-27 01:36:34,130,96.566667,2022-09-02 01:45:52,2022-11-27 01:45:52,2108348827,chevrolet,157


In [ ]:
df_lic[['Time']] = df_lic[['Time']].astype(float)
# #.apply(lambda x: x.value)
print(df_lic.dtypes)
df_lic.head()

In [ ]:
df['time2'] = df['Time'].astype(int).str.split().str[1]
# df['time2'] = df['Time'].dt.strftime('%H:%M:%S')
df.tail(5)

In [ ]:
time2=df_lic["t1_indexed"]/pd
time2.dtype
# df['timedelta_column'] / pd

In [ ]:
df_lic['t1_indexed'] = pd.to_datetime(df_lic['t1_indexed'].view(int) // 10 ** 9)
# #.apply(lambda x: x.value)
# pd.to_datetime(df['time'], unit='s')
df_lic.dtypes

In [ ]:
df2.head()

In [ ]:
df1.head()

In [ ]:
print(df2.dtypes)
# print(df1.dtypes)

In [ ]:
df1.sort_values(by='t1_indexed', inplace=True)
df2.sort_values(by='t1_indexed', inplace=True)

In [ ]:
df3= pd.merge_asof(
    df1, df2, on="t1_indexed",direction="nearest",  
        allow_exact_matches=False)

df3.head()

In [ ]:
df3.to_csv('./Results2/merged_BC_TS_index.csv')

In [ ]:
print(df_CO2.dtypes)
print(df_BC.dtypes)

In [ ]:
# df_CO2.sort_values(by='t1_indexed', inplace=True)
# df_BC.sort_values(by='t1_indexed', inplace=True)

In [ ]:
df_BC_CO2= pd.merge_asof(
    df_CO2, df_BC, on="t1_indexed",direction="nearest",  
        allow_exact_matches=False)

df_BC_CO2.head()



In [ ]:
df_BC_CO2.to_csv('./Results2/merged_CO2_BC_indexed.csv')

In [ ]:
df_CO2 = pd.read_csv('./Results2/merged_CO2_BC_indexed.csv', index_col=0)
df_lic = pd.read_csv('./Results2/simplified_ALPR.csv', index_col=0)
print(df_CO2_BC.head())
print(df_lic.head)